# Busca (Parte III)

## Mochila com Número Irrestrito de Itens com Valor

In [1]:
max_size = 19
items = [(1,3),(4,6),(5,7)]

Tamanho de Estados

In [2]:
def state_size(state, items):
    size = 0
    for i in range(len(state)):
        size += state[i] * items[i][1]
    return size

In [3]:
state_size([1,1,1], items)

16

Avaliação de Estados

In [4]:
def evaluate_state(state, items):
    value = 0
    for i in range(len(state)):
        value += state[i] * items[i][0]
    return value

In [5]:
evaluate_state([1,2,0], items)

9

Geração de Estados

In [6]:
def generate_states(initial_state):
    states = []
    for i in range (len(initial_state)):
        aux = initial_state.copy()
        aux[i] = initial_state[i] + 1
        states.append(aux)
    return states

In [7]:
generate_states([0,0,0])

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

Roleta

Soma de valores de todos os estados

In [8]:
def states_total_value(states):
    total_sum = 0
    for state in states:
        total_sum = total_sum + state[0]
    return total_sum

In [9]:
states_total_value([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

10

Construção da roleta

In [10]:
def roulette_construction(states):
    aux_states = []
    roulette = []
    total_value = states_total_value(states)

    for state in states:
        value = state[0]
        if total_value != 0:
            ratio = value/total_value
        else:
            ratio = 1
        aux_states.append((ratio,state[1]))
 
    acc_value = 0
    for state in aux_states:
        acc_value = acc_value + state[0]
        s = (acc_value,state[1])
        roulette.append(s)
    return roulette

In [11]:
roulette_construction([(1, [1,0,0]), (4, [0, 1, 0]), (5, [0, 0, 1])])

[(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])]

Rodar a roleta

In [12]:
import random

def roulette_run (rounds, roulette):
    if roulette == []:
        return []
    selected = []
    while len(selected) < rounds:
        r = random.uniform(0,1)
        for state in roulette:
            if r <= state[0]:
                selected.append(state[1])
                break
    return selected

In [13]:
roulette_run (1, [(0.1, [1, 0, 0]), (0.5, [0, 1, 0]), (1.0, [0, 0, 1])])

[[0, 0, 1]]

Estado Inicial Aleatório

In [14]:
import random

def generate_initial_state(max_size, items):
    initial_state = []
    for i in range(len(items)):
        initial_state.append(random.randint(0,max_size//items[i][1]))
    if state_size(initial_state,items) > max_size:
        while True:
            for i in range(len(items)):
                if initial_state[i] > 0:
                    initial_state[i] = initial_state[i] - 1
                    if state_size(initial_state,items) <= max_size:
                        return initial_state
    return initial_state

In [15]:
generate_initial_state(max_size, items)

[3, 1, 0]

Primeiro elemento de uma lista ou tupla

In [16]:
def first(x):
    return x[0]

Operador de Seleção

In [17]:
def selection(value_population,n):
    aux_population = roulette_construction(value_population)
    new_population = roulette_run(n, aux_population)
    return new_population

In [18]:
pop = [(1, [1, 0, 0]), (4, [0, 1, 0]), (5, [0, 0, 1]), (5, [1, 1, 0]), (6, [1, 0, 1]), (10, [1, 1, 1])]
selection (pop, 3)

[[1, 1, 1], [1, 1, 0], [1, 0, 1]]

Operador de Recombinação

In [19]:
def crossover(dad,mom):
    r = random.randint(0, len(dad) - 1)
    son = dad[:r]+mom[r:]
    daug = mom[:r]+dad[r:]
    return son, daug

In [20]:
crossover([0,0,0,0,0,0],[1,1,1,1,1,1])

([0, 0, 0, 1, 1, 1], [1, 1, 1, 0, 0, 0])

Operador de Mutação

In [21]:
def mutation (indiv):
    individual = indiv.copy()
    rand = random.randint(0, len(individual) - 1)
    if individual[rand] > 0:
        r = random.uniform(0,1)
        if r > 0.5:
            individual[rand] = individual[rand] + 1
        else:
            individual[rand] = individual[rand] - 1
    else:
        individual[rand] = individual[rand] + 1
        
    return individual

In [22]:
mutation ([1,2,1,0,1,3])

[1, 3, 1, 0, 1, 3]

População Inicial

In [23]:
def initial_population(n, max_size, items):
    pop = []
    count = 0
    while count < n:
        individual = generate_initial_state(max_size, items)
        pop = pop + [individual]
        count += 1
    return pop

In [24]:
initial_population(5, max_size, items)

[[3, 0, 1], [3, 1, 0], [0, 1, 0], [4, 1, 0], [1, 2, 0]]

Convergência

In [25]:
def convergent(population):
    conv = False
    if population != []:
        base = population[0]
        i = 0
        while i < len(population):
            if base != population[i]:
                return False
            i += 1
        return True

In [26]:
convergent([[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]])

True

Avaliação

In [27]:
def evaluate_population (pop, items):
    eval = []
    for s in pop:
        eval = eval + [(evaluate_state(s, items), s)]
    return eval        

In [28]:
evaluate_population ([[1, 0, 1], [0, 1, 1], [0, 0, 1]], items)

[(6, [1, 0, 1]), (9, [0, 1, 1]), (5, [0, 0, 1])]

Elitismo

In [29]:
import math

def elitism (val_pop, pct):
    n = math.floor((pct/100)*len(val_pop))
    if n < 1:
        n = 1
    val_elite = sorted (val_pop, key = first, reverse = True)[:n]
    elite = [s for v,s in val_elite]
    return elite

In [30]:
elitism ([(6, [1, 0, 1]), (9, [0, 1, 1]), (5, [0, 0, 1])], 70)

[[0, 1, 1], [1, 0, 1]]

Etapa de Recombinação

In [31]:
def crossover_step (population, crossover_ratio, max_size, items):
    new_pop = []
    
    for _ in range (round(len(population)/2)):
        rand = random.uniform(0, 1)
        fst_ind = random.randint(0, len(population) - 1)
        scd_ind = random.randint(0, len(population) - 1)
        parent1 = population[fst_ind] 
        parent2 = population[scd_ind]

        if rand <= crossover_ratio:
            offspring1, offspring2 = crossover(parent1, parent2)            
            if state_size(offspring1, items) > max_size:
                offspring1 = parent1
            if state_size(offspring2, items) > max_size:
                offspring2 = parent2
        else:
            offspring1, offspring2 = parent1, parent2
                
        new_pop = new_pop + [offspring1, offspring2]
        
    return new_pop

In [32]:
crossover_step ([[0, 2, 1], [1, 1, 1], [1, 2, 0], [3, 0, 1]], 0.8, max_size, items)

[[0, 2, 1], [1, 1, 1], [1, 2, 0], [0, 2, 1]]

Etapa de Mutação

In [33]:
def mutation_step (population, mutation_ratio, max_size, items):
    ind = 0
    for individual in population:
        rand = random.uniform(0, 1)

        if rand <= mutation_ratio:
            mutated = mutation(individual)
            if state_size(mutated, items) <= max_size:
                population[ind] = mutated
                
        ind+=1
        
    return population   

In [34]:
mutation_step ([[0, 2, 1], [1, 1, 1], [1, 2, 0], [3, 0, 1]], 0.8, max_size, items)

[[0, 2, 1], [1, 1, 1], [1, 2, 0], [3, 0, 0]]

Algoritmo Genético

In [35]:
import time

def genetic (max_size, items, pop_size, max_iter, cross_ratio, mut_ratio, max_time, elite_pct):
# mut_ratio, max_time, elite_pct

    start = time.process_time()
    opt_state = [0] * len(items)
    opt_value = 0
    pop = initial_population(pop_size, max_size, items)
    conv = convergent(pop)
    iter = 0    
    end = 0

    while not conv and iter < max_iter and end-start <= max_time:
        
        val_pop = evaluate_population (pop, items)
        new_pop = elitism (val_pop, elite_pct)
        best = new_pop[0]
        val_best = evaluate_state(best, items)

        if (val_best > opt_value):
            opt_state = best
            opt_value = val_best

        selected = selection(val_pop, pop_size - len(new_pop)) 
        crossed = crossover_step(selected, cross_ratio, max_size, items)
        mutated = mutation_step(crossed, mut_ratio, max_size, items)
        pop = new_pop + mutated
        conv = convergent(pop)
        iter+=1
        end = time.process_time()
        
  
    return opt_state, state_size(opt_state, items), opt_value, iter, conv


In [36]:
genetic (max_size, items, 100, 200, 0.9, 0.1, 120, 20)

([0, 2, 1], 19, 13, 44, True)

In [37]:
complex_items = [(109, 130), (104, 267), (42, 71), (48, 125), (85, 190), (36, 55), (18, 48), (43, 132), (21, 69), (108, 450), (39, 71), (125, 348), (53, 193), (51, 97), (102, 158), (131, 258), (143, 433), (119, 267), (23, 89), (106, 348), (66, 174), (31, 81), (123, 372), (127, 176), (115, 419), (32, 130), (11, 22), (104, 453), (29, 126), (140, 384), (29, 82), (120, 286), (82, 188), (144, 425), (21, 83), (142, 567), (10, 14), (94, 166), (131, 173), (39, 113), (123, 428), (33, 83), (50, 217), (24, 52), (118, 471), (67, 277), (67, 252), (36, 124), (36, 104), (43, 168), (42, 173), (129, 423), (68, 238), (26, 69), (74, 261), (73, 258), (85, 215), (130, 174), (110, 348), (26, 75), (69, 198), (58, 210), (73, 193), (129, 393), (97, 160), (113, 348), (60, 209), (25, 64), (83, 334), (93, 101), (68, 184), (100, 436), (30, 100), (55, 155), (146, 613), (24, 68), (15, 34), (107, 273), (149, 427), (147, 611), (45, 151), (28, 121), (98, 228), (35, 84), (54, 132), (35, 75), (82, 317), (26, 37), (51, 132), (101, 373), (36, 82), (124, 528), (79, 112), (144, 293), (24, 27), (59, 264), (75, 84), (14, 45), (46, 93), (66, 115)]

complex_max_size = 10000

genetic (complex_max_size, complex_items, 100, 200, 0.9, 0.1, 120, 20)

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  332,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  195,
  0,
  0,
  0,
  0,
  0],
 9987,
 8052,
 20,
 True)

PSO

In [141]:
from math import floor

"""def distance (x, y): #euclidean
    sum = 0
    for i in range(len(x)):
        dif = x[i]-y[i]
        sum += dif*dif
    return sqrt (sum)"""

class Particle:
    w = 1
    c1 = 1.2
    c2 = 1.2
    
    def __init__(self, max_size, items):
        self.pos = generate_initial_state(max_size, items)
        self.vel = [0] * len(items)
        self.opt = self.pos
        self.opt_value = evaluate_state(self.opt, items) 

    def update_vel(self, best):
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        for i in range(len(self.vel)):
            self.vel[i] = floor(Particle.w*self.vel[i] + Particle.c1*r1*(abs(self.opt[i] - self.pos[i])) + 
                                Particle.c2*r2*(abs(best[i] - self.pos[i])))
        
    def update_pos(self, max_size, items):
        for i in range(len(self.pos)):
            self.pos[i] += self.vel[i] 
            if state_size(self.pos, items) > max_size:
                self.pos[i] -= self.vel[i]
                return
            
    def update_local_optima(self, items):
        eval_particle = evaluate_state(self.pos, items)
        if eval_particle > self.opt_value:
            self.opt = self.pos
            self.opt_value = eval_particle 

    def get_pos(self):
        return self.pos
            
    def get_local_optima(self):
        return self.opt, self.opt_value

In [146]:
class Swarm:

    
    def __init__(self, ms, its, ps):
        Swarm.max_size = ms
        Swarm.items = its
        Swarm.pop_size = ps
        self.best = [0] * len(items)
        self.best_val = 0
        self.pop = []
        for i in range(Swarm.pop_size):
            prtc = Particle(Swarm.max_size, Swarm.items)
            particle_value = evaluate_state(prtc.get_pos(), Swarm.items)
            self.pop.append(prtc)
            if particle_value > self.best_val:
                self.best = prtc.get_pos()
                self.best_val = particle_value
                
    def update_swarm(self):
        for i in range(Swarm.pop_size):
            self.pop[i].update_vel(self.best)
            self.pop[i].update_pos(Swarm.max_size, Swarm.items)
            self.pop[i].update_local_optima(Swarm.items)
            
    def update_global_optima(self):
        for i in range(Swarm.pop_size):
            s, v = self.pop[i].get_local_optima()
            if v > self.best_val:
                self.best = s
                self.best_val = v
                
    def get_population(self):
        popul = []
        for i in range(Swarm.pop_size):
            popul.append(self.pop[i].get_pos())
        return popul             
                
    def get_global_optima(self):
        return self.best, self.best_val

In [149]:
def pso (max_size, items, pop_size, max_iter, max_time):
    start = time.process_time()
    itera = 0    
    end = 0
    pop = Swarm(max_size, items, pop_size)
    
    conv = convergent(pop.get_population())

    while not conv and itera < max_iter and end-start <= max_time:   
               
        pop.update_swarm()
        pop.update_global_optima()
        conv = convergent(pop.get_population())
        itera+=1
        end = time.process_time()
    
    best, best_val = pop.get_global_optima()
    return best, state_size(best, items), best_val, itera, conv   
        

In [150]:
pso (max_size, items, 100, 200, 120)

([0, 2, 1], 19, 13, 200, False)

In [151]:
pso (complex_max_size, complex_items, 100, 200, 120)

([0,
  0,
  0,
  0,
  0,
  38,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  120,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  224,
  0,
  0,
  4,
  0,
  0],
 9998,
 8000,
 200,
 False)

### Exercícios de Fixação

1. Considere o problema da mochila irrestrita com tamanho de mochila igual a 1381 e com a seguinte lista de pares (valor, tamanho) de itens {(1,3),(4,6),(5,7),(3,4),(2,6),(2,3),(6,8),(1,2),(3,5),(7,10),(10,15),(13,20),(24,25),(29,37)}. Execute várias vezes o algoritmo genético e o método grasp feito na aula anterior. Reporte o melhor resultado encontrado e o resultado médio obtido eplos dois algoritmos nas várias execuções. Qual o método que obteve os melhores resultados? Tente ajustar os hiperparâmetros do genético para ver se consegue melhorar seus resultados.

2. Explique o que é elitismo e porque ele deve ser usado de forma cautelosa? Rode várias vezes o algoritmo genético no problema da questão 1 com percentual de elitismo igual a 50%. Observe se o resultado final obteve convergência. Repita com o percentual igual a 1%. Compare em termos de convergência e resultados obtidos.

3. Altere a implementação do operador de crossover do algoritmo genético para transformá-lo em um operador de segregação, isto é, para decidir aleatoriamente qual filho receberá o gene de cada pai.

4. Liste as diferenças entre o algoritmo genético apresentado nesta aula e o constante no artigo "A Survey of Metaheuristics Methods for Bioinformatics Applications"

5. O artigo "Soft Computing Classifier Ensemble for Fault Diagnosis" usa várias metaheurísticas para a construção de combinados de classificadores aplicados ao diagnóstico de falhas em motobombas. Indique quais metaheurísticas foram usadas e para quê cada uma  delas foi aplicada no trabalho.